<h1>Notation</h1>
<ul>
    <li>F - State Transition Matrix</li>
    <li>G - Process Noise Transformation</li>
    <li>H - Observation Matrix(State Tracks more variables than what is measured)</li>
    <li>R - Measurement Noise Covariance</li>
    <li>Q - Process Noise Covariance</li>
    <li>x0,P - Initial state and state covariance</li>
    <li> C - Weighted Adjacencey Matrix</li>
</ul>

In [1]:
class Node:
    def __init__(self,id,F,G,H,R,Q,x0,P):
        self.id = id
        self.F = F
        self.G = G
        self.H = H
        self.R = R
        self.Q = Q

        #Has an instance for each node
        self.x = x0
        self.n = len(x0)

        self.P = P

        self.nbhrs = []
        self.nbhr_weights = {}

        self.psi = np.copy(x0)
    
    def predict(self):
        self.x = self.F@(np.sum([self.nbhr_weights[node.id]*node.psi for node in self.nbhrs],0))
        self.P = (self.F@self.P@self.F.T) + (self.G@self.Q@self.G.T)

    def update(self, y):
        S = lambda node: (node.H @self.P @ node.H.T) + node.R
        #Original Code for K
        #K = {node.id: self.P@ node.H.T @ np.linalg.inv(S(node)) for node in self.nbhrs}
        #We use a solver to avoid inverses:
        #K[i]@S = P@H[i].T --> S.T@K[i].T = H[i]@P.T --> cholesky solve for K[i]
        K = {node.id:sp.linalg.cho_solve(sp.linalg.cho_factor(S(node).T),(node.H @ self.P.T)).T for node in self.nbhrs}
        
        I = lambda node: y[node.id] - (node.H @self.psi)
        self.psi = self.x

        for node in self.nbhrs:
            self.psi = self.psi+(K[node.id]@I(node))
        
        for node in self.nbhrs:
            self.P = (np.eye(self.n,self.n) - K[node.id]@node.H)@self.P


class DiffKF:
    def __init__(self,C,F,G,H,R,Q,x0,P):
        
        #Number of nodes
        self.n = len(x0)

        #weighted adjacencey matrix, nodes must be connected to themselves
        self.C = C

        self.nodes = []
        for i in range(self.n):
            self.nodes.append(Node(i,F[i],G[i],H[i],R[i],Q[i],x0[i],P[i]))
        
        for i in range(self.n):
            for j in range(self.n):
                if self.C[i][j] != 0:
                    self.nodes[i].nbhrs.append(self.nodes[j])
                    self.nodes[i].nbhr_weights[j] = C[i][j]
    
    def predict(self): #a.k.a diffusion update
        result = []
        for node in self.nodes:
            node.predict()
            result.append(node.x)
        
        return result
        
    
    def update(self, y): #a.k.a Incremental update
        for i,node in enumerate(self.nodes):
            node.update(y)